In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import os

from os.path import dirname



root_path = dirname(os.getcwd()) + "/HGNN_NA"

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI_Challenge_2012_W_Complete"

In [3]:
categorical_columns = [
    'org:resource', 'Activity',
    "org:resource:role"
]
real_value_columns = [
"time:timestamp", 'case:AMOUNT_REQ', 'case:REG_DATE'
]

In [17]:
tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")
tab_all.head()

/tmp/ipykernel_185421/3535436184.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")


,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ,org:resource:role
0,10912.0,COMPLETE,W_Afhandelen leads,10.442930,10.421865,173712,10.308953,Role 1
1,11019.0,COMPLETE,W_Completeren aanvraag,10.707527,10.421865,173712,10.308953,Role 1
2,11180.0,COMPLETE,W_Completeren aanvraag,12.260206,10.421865,173712,10.308953,Role 1
3,11180.0,COMPLETE,W_Completeren aanvraag,12.316947,10.421865,173712,10.308953,Role 1
4,10912.0,COMPLETE,W_Afhandelen leads,10.454063,10.398580,173706,9.798127,Role 1


In [5]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [16]:
with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_BOOOH.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_BOOOH.pkl", "rb") as f:
    X_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_BOOOH.pkl", "rb") as f:
    X_test = pickle.load(f)

KeyboardInterrupt: 

In [ ]:
for i in range(len(X_train)):
    if 'Activity' not in X_train[i].x_dict.keys():
        print("aaa")    
for i in range(len(X_valid)):
    if 'Activity' not in X_valid[i].x_dict.keys():
        print("bbbb")   
for i in range(len(X_test)):
    if 'Activity' not in X_test[i].x_dict.keys():
        print("cccc")           

In [ ]:
len(X_train)

In [ ]:
X_train = X_train[:int(len(X_train)*0.8)]

In [7]:

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToUndirected, NormalizeFeatures

transform = ToUndirected()

with torch.no_grad():
        for i in range(len(X_train)):
                X_train[i] = transform(X_train[i])
        for i in range(len(X_valid)):
                X_valid[i] = transform(X_valid[i])
        for i in range(len(X_test)):
                X_test[i] = transform(X_test[i])
    






In [8]:
edge_types = set()
node_types = set()
for i in range(len(X_train)):
    n, edge_type = X_train[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_valid)):
    n, edge_type = X_valid[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_test)):
    n, edge_type = X_test[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)



In [9]:
node_types = list(node_types)
edge_types = list(edge_types)

In [ ]:
node_types

In [ ]:
edge_types

In [10]:
def get_weights(load, key):
    weights = []
    
    cl_train = [0 for _ in tab_all[key].unique()]
    
    print(cl_train)
    
    for i,x in enumerate(load):

        
        classes = x.y[key]

        # print(classes)
        
        for c in list(classes):
            try:
                cl_train[c] +=1
            except KeyError:
                cl_train[c] = 1
    s = sum(cl_train)
    
    print(cl_train)
    
    weights = [s/x if x > 0 else 0 for x in cl_train]

    # weights = [0.7,0.7,1,0.7,0.7,0.7,0.7,0.7,0.7,0.7]
    weights = torch.tensor(weights, device=device)
    print(weights)
    return weights
    

In [11]:
edge_features_dims = {}

for i in range(len(X_train)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_valid)):
    for k in X_valid[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_valid[i].edge_attr_dict[k].shape[1]
for i in range(len(X_test)):
    for k in X_test[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_test[i].edge_attr_dict[k].shape[1]

edge_features_dims

{('org:resource', 'related_to', 'org:resource'): 2,
 ('Activity', 'followed_by', 'Activity'): 2,
 ('time:timestamp', 'related_to', 'time:timestamp'): 2,
 ('org:resource:role', 'related_to', 'org:resource:role'): 2}

In [12]:
act_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "Activity")
# res_roles_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "org:resource:role")


[0, 0, 0, 0, 0, 0]
[172, 66, 7218, 6261, 12112, 4087]
tensor([173.9302, 453.2727,   4.1446,   4.7782,   2.4699,   7.3198],
       device='cuda:0')


KeyError: nan

In [19]:
tab_all["org:resource"] = [str(x) for x in tab_all["org:resource"].values]

In [20]:
from data import utils

one_hot_resource = utils.get_one_hot_encoder(tab_all, "org:resource")
one_hot_resource_role = utils.get_one_hot_encoder(tab_all, "org:resource:role")
resources = tab_all["org:resource"]
resources_role = tab_all["org:resource:role"]
map_resource_to_role = {}
for i in range(len(resources)):
    map_resource_to_role[resources[i]] = resources_role[i]

map_resource_to_role

{'10912.0': 'Role 1',
 '11019.0': 'Role 1',
 '11180.0': 'Role 1',
 'NAN': 'Role 1',
 '10982.0': 'Role 1',
 '11002.0': 'Role 1',
 '11049.0': 'Role 1',
 '10629.0': 'Role 2',
 '11122.0': 'Role 1',
 '10913.0': 'Role 1',
 '10889.0': 'Role 1',
 '10972.0': 'Role 2',
 '11121.0': 'Role 1',
 '10939.0': 'Role 1',
 '10609.0': 'Role 2',
 '11009.0': 'Role 1',
 '11201.0': 'Role 1',
 '11119.0': 'Role 1',
 '10861.0': 'Role 1',
 '11203.0': 'Role 1',
 '11181.0': 'Role 1',
 '11189.0': 'Role 1',
 '10809.0': 'Role 2',
 '10899.0': 'Role 1',
 '10138.0': 'Role 2',
 '11000.0': 'Role 1',
 '10863.0': 'Role 1',
 '11169.0': 'Role 1',
 '11179.0': 'Role 1',
 '11001.0': 'Role 1',
 '10228.0': 'Role 1',
 '10909.0': 'Role 1',
 '10789.0': 'Role 1',
 '10881.0': 'Role 1',
 '10910.0': 'Role 1',
 '10929.0': 'Role 1',
 '10931.0': 'Role 1',
 '11259.0': 'Role 1',
 '10188.0': 'Role 3',
 '10779.0': 'Role 1',
 '10914.0': 'Role 1',
 '11339.0': 'Role 2',
 '10933.0': 'Role 1',
 '11079.0': 'Role 1',
 '10932.0': 'Role 1',
 '10935.0': 'R

In [21]:
map_resource_index = {}
for x in tab_all["org:resource"].unique():
    map_resource_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource, np.array([x])))).item()
map_resource_index

{'10912.0': 18,
 '11019.0': 34,
 '11180.0': 42,
 'NAN': 59,
 '10982.0': 28,
 '11002.0': 31,
 '11049.0': 35,
 '10629.0': 6,
 '11122.0': 39,
 '10913.0': 19,
 '10889.0': 14,
 '10972.0': 27,
 '11121.0': 38,
 '10939.0': 26,
 '10609.0': 5,
 '11009.0': 33,
 '11201.0': 46,
 '11119.0': 37,
 '10861.0': 11,
 '11203.0': 47,
 '11181.0': 43,
 '11189.0': 44,
 '10809.0': 9,
 '10899.0': 15,
 '10138.0': 2,
 '11000.0': 29,
 '10863.0': 12,
 '11169.0': 40,
 '11179.0': 41,
 '11001.0': 30,
 '10228.0': 4,
 '10909.0': 16,
 '10789.0': 8,
 '10881.0': 13,
 '10910.0': 17,
 '10929.0': 21,
 '10931.0': 22,
 '11259.0': 49,
 '10188.0': 3,
 '10779.0': 7,
 '10914.0': 20,
 '11339.0': 58,
 '10933.0': 24,
 '11079.0': 36,
 '10932.0': 23,
 '10935.0': 25,
 '11254.0': 48,
 '11003.0': 32,
 '11269.0': 50,
 '10821.0': 10,
 '11289.0': 51,
 '10125.0': 1,
 '112.0': 45,
 '11299.0': 52,
 '10124.0': 0,
 '11309.0': 56,
 '11300.0': 53,
 '11302.0': 54,
 '11319.0': 57,
 '11304.0': 55}

In [22]:
map_resource_role_index = {}
for x in tab_all["org:resource:role"].unique():
    map_resource_role_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource_role, np.array([x])))).item()
map_resource_role_index

{'Role 1': 0, 'Role 2': 1, 'Role 3': 2}

In [23]:
map_resource_to_role = { map_resource_index[k] : map_resource_role_index[v] for k,v in map_resource_to_role.items()}
map_resource_to_role

{18: 0,
 34: 0,
 42: 0,
 59: 0,
 28: 0,
 31: 0,
 35: 0,
 6: 1,
 39: 0,
 19: 0,
 14: 0,
 27: 1,
 38: 0,
 26: 0,
 5: 1,
 33: 0,
 46: 0,
 37: 0,
 11: 0,
 47: 0,
 43: 0,
 44: 0,
 9: 1,
 15: 0,
 2: 1,
 29: 0,
 12: 0,
 40: 0,
 41: 0,
 30: 0,
 4: 0,
 16: 0,
 8: 0,
 13: 0,
 17: 0,
 21: 0,
 22: 0,
 49: 0,
 3: 2,
 7: 0,
 20: 0,
 58: 1,
 24: 0,
 36: 0,
 23: 0,
 25: 0,
 48: 0,
 32: 0,
 50: 0,
 10: 1,
 51: 1,
 1: 1,
 45: 0,
 52: 0,
 0: 0,
 56: 0,
 53: 0,
 54: 0,
 57: 0,
 55: 2}

## Hyperopt

In [24]:
from ax.service.managed_loop import optimize

In [25]:
from torch_geometric.nn import (
    HeteroConv,
    global_mean_pool,
    GATv2Conv
)
from torch.nn import (
    ModuleList,
    Module,
    Linear
  )
from typing_extensions import Self

In [26]:
from copy import deepcopy


class HGNN(Module):

    def __init__(self, output_cat, output_real,nodes_relations, relations_with_features, parameters) -> Self:  # type: ignore
        super().__init__()

        # List of convolutional layers
        
        hid = parameters["hid"]
        layers = parameters["layers"]
        aggregation = parameters["aggregation"]
        n_heads = parameters["heads"]
        
        self.output_cat = output_cat
        self.output_real = output_real
        
        self.convs = ModuleList()
        for _ in range(layers):
            conv = HeteroConv(
                {
                    relation: (
                        GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid, heads=n_heads, concat=False)
                        if relation not in relations_with_features
                        else GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid,heads=n_heads, edge_dim=relations_with_features[relation], concat=False)
                    )
                    for relation in nodes_relations
                },
                aggr=aggregation,
            )

            self.convs.append(conv)

        self.FC = {}
        
        for k in output_cat:
            self.FC[k] = Linear(hid, output_cat[k], device=device)
        for k in output_real:
            self.FC[k] = Linear(hid, 1, device=device)
        
    
        
        

    def forward(self, batch):

        #print(f"inside: {batch.x_dict.keys()}")
        
       
        
        for i in range(len(self.convs)):
            batch.x_dict = self.convs[i]( 
                batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
            )
            #print(f"inside2_{i}: {batch.x_dict.keys()}")
            batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}
        
        #print(batch.x_dict)
        #batch.x_dict = self.convs[0]( 
        #      batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
        #)
        #batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}
        #print(f"inside0: {batch.x_dict.keys()}")
        #print(batch.x_dict)
        
        #batch.x_dict = self.convs[1]( 
        #      batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
        #)
        #batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}
        #print(f"inside1: {batch.x_dict.keys()}")
        #print(batch.x_dict)
        
        

        output = {}
        try:
            for k in self.output_cat:
                output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
                output[k] = self.FC[k](output[k])
            for k in self.output_real:
                output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
                output[k] = self.FC[k](output[k]).reshape(1,-1)[0]
        except KeyError:
            print("error")
            return None   

        return output

In [27]:
from torcheval.metrics.functional import multiclass_accuracy, multiclass_f1_score

In [28]:
import torch.nn as nn

In [29]:
import time

In [30]:
from copy import deepcopy


def train_hgnn(config, output_cat, output_real, epochs=15):
    print(config)

    net = HGNN(
        parameters=config,
        output_cat=output_cat,
        output_real=output_real,
        nodes_relations=edge_types,
        relations_with_features=edge_features_dims,
    )
    net = net.to(device)

    losses = {}

    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()

    train_loader = DataLoader(X_train, batch_size=config["batch_size"], shuffle=False)
    valid_loader = DataLoader(X_valid, batch_size=config["batch_size"], shuffle=False)

    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    best_model = None
    best_loss = 0
    patience = 4
    pat_count = 0

    torch.cuda.empty_cache()

    for epoch in range(0, epochs):
        start_time = time.time()

        print(f"Epoch: {epoch}\n")

        net.train()
        
        #for i,x in enumerate(train_loader):
        #    if "Activity" not in x.x_dict.keys():
        #        print("aa")
        
        for _, x in enumerate(train_loader):
            x = x.to(device)

            labels = x.y

            optimizer.zero_grad()
            #print("Keys in x_dict:", x.x_dict.keys())
        
            outputs = net(x)
            
            if outputs != None:
                

                losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

                total_loss = 0
                for k in losses_step:
                    total_loss += losses_step[k]

                total_loss.backward()
                optimizer.step()
            

        predictions_categorical = {k: [] for k in output_cat}
        target_categorical = {k: [] for k in output_cat}

        avg_MAE = {k: [] for k in output_real}

        running_total_loss = []
        
        net.eval()
        with torch.no_grad():
            for i, x in enumerate(valid_loader):
                x = x.to(device)

                labels = x.y

                outputs = net(x)

                if outputs != None:
                
                    losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}
    
                    running_total_loss.append(sum(list(losses_step.values())))
    
                    for k in output_cat:
                        predictions_categorical[k].append(
                            torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                        )
                        target_categorical[k].append(labels[k])
    
                    for k in output_real:
                        avg_MAE[k].append(losses_step[k])
    
        for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])

        macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )

        accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }

        avg_MAE = {k: sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}

        val_loss = sum(running_total_loss) / len(running_total_loss)

        print(f"\nVALIDATION")
        for k in accuracy:
            (
                print("{}: acc {:.4f}".format(k, accuracy[k]))
                if k != "Activity"
                else print(
                    "{}: acc {:.4f} macroF1 {:.4f}".format(
                        k, accuracy[k], macro_f1_activity.item()
                    )
                )
            )
        for k in avg_MAE:
            print("{}: MAE {:.4f}".format(k, avg_MAE[k]))
        print("TOTAL_LOSS: {:.4f}".format(val_loss))
        print("epoch time {}s\n".format(time.time() - start_time))

        if epoch == 0:
            best_model = deepcopy(net)
            best_loss = val_loss
        else:
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = deepcopy(net)
                pat_count = 0
                print("new best model found")
            if pat_count == patience:
                print(
                    "Validation performance didn't improve for {} epochs. Training stops.".format(
                        pat_count
                    )
                )
                return best_model
        pat_count += 1

    return best_model

In [31]:
def test_hgnn(net, output_cat, output_real):
    test_loader = DataLoader(X_train, batch_size=128, shuffle=False)
    
    losses = {}
    
    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()
    
    
    
    
    
    predictions_categorical = {k: [] for k in output_cat}
    target_categorical = {k: [] for k in output_cat}

    avg_MAE = {k : [] for k in output_real}
    
    total_loss = []
        
    net.eval()
    with torch.no_grad():
        for i, x in enumerate(test_loader):
            x = x.to(device)
            
            labels = x.y
            
            outputs = net(x)
            
     
            losses_step = {k: losses[k](outputs[k], labels[k]).item() for k in losses}
            total_loss.append(sum(list(losses_step.values())))
            
            for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])
            
            
            for k in output_real:
                    avg_MAE[k].append(losses_step[k])
                    
    for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])
               
            
    macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )
            
    accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }
    
    resource_to_role_acc = multiclass_accuracy(
        torch.tensor([map_resource_to_role[x.item()] for x in predictions_categorical["org:resource"]], device=device),
        target_categorical["org:resource:role"],
        num_classes=output_cat["org:resource:role"]
    )
    
    avg_MAE = {k : sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}
    
    
    Average_total_loss = sum(total_loss) / len(total_loss)
    
    res = {f"{k}_acc" : accuracy[k].item() for k in accuracy} | {"Resource_to_role_acc" : resource_to_role_acc.item()} | {"MacroF1Act" : macro_f1_activity.item()} | {f"{k}_mae" : avg_MAE[k] for k in avg_MAE} | {"AVG_total_loss" : Average_total_loss} 
    
    print(res)
    
    return res

In [ ]:
train_loader = DataLoader(X_train, batch_size=256, shuffle=True)
for i,x in enumerate(train_loader):
    if "Activity" not in x.x_dict.keys():
        print("aa")

In [32]:
list_unique = {k : list(tab_all[k].unique()) for k in categorical_columns}
list_unique

{'org:resource': ['10912.0',
  '11019.0',
  '11180.0',
  'NAN',
  '10982.0',
  '11002.0',
  '11049.0',
  '10629.0',
  '11122.0',
  '10913.0',
  '10889.0',
  '10972.0',
  '11121.0',
  '10939.0',
  '10609.0',
  '11009.0',
  '11201.0',
  '11119.0',
  '10861.0',
  '11203.0',
  '11181.0',
  '11189.0',
  '10809.0',
  '10899.0',
  '10138.0',
  '11000.0',
  '10863.0',
  '11169.0',
  '11179.0',
  '11001.0',
  '10228.0',
  '10909.0',
  '10789.0',
  '10881.0',
  '10910.0',
  '10929.0',
  '10931.0',
  '11259.0',
  '10188.0',
  '10779.0',
  '10914.0',
  '11339.0',
  '10933.0',
  '11079.0',
  '10932.0',
  '10935.0',
  '11254.0',
  '11003.0',
  '11269.0',
  '10821.0',
  '11289.0',
  '10125.0',
  '112.0',
  '11299.0',
  '10124.0',
  '11309.0',
  '11300.0',
  '11302.0',
  '11319.0',
  '11304.0'],
 'Activity': ['W_Afhandelen leads',
  'W_Completeren aanvraag',
  'W_Nabellen offertes',
  'W_Valideren aanvraag',
  'W_Nabellen incomplete dossiers',
  'W_Beoordelen fraude'],
 'org:resource:role': ['Role 1',

In [38]:
outputcat = {k : len(list_unique[k]) if len(list_unique[k]) > 1 else None for k in list_unique}
outputcat = {k : v for k,v in outputcat.items() if v != None}
outputreal = ['time:timestamp']
print(outputcat)
print(outputreal)

{'org:resource': 60, 'Activity': 6, 'org:resource:role': 3}
['time:timestamp']


In [39]:
def train_evaluate(config):
    trained_net = train_hgnn(config, output_cat=outputcat, output_real=outputreal, epochs = 1)
    return test_hgnn(trained_net, output_cat=outputcat, output_real=outputreal)

In [44]:
config = {
    "lr" : 0.001903641203982847,
    "heads" : 1,
    "hid" : 128, 
    "layers" : 2,
    "batch_size" : 2048,
    "aggregation" : "max"
}

In [45]:
train_evaluate(config)

{'lr': 0.001903641203982847, 'heads': 1, 'hid': 128, 'layers': 2, 'batch_size': 2048, 'aggregation': 'max'}
Epoch: 0


VALIDATION
org:resource: acc 0.0431
Activity: acc 0.3604 macroF1 0.0883
org:resource:role: acc 0.8282
time:timestamp: MAE 1.9208
TOTAL_LOSS: 8.2149
epoch time 32.81581926345825s



KeyboardInterrupt: 

In [46]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "hid", "type": "choice", "values": [128], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "layers", "type": "choice", "values": [2, 3, 4, 5], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "layers", "type": "choice", "values": [2], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "lr", "type": "range", "bounds": [1e-4, 1e-1], "value_type": "float", "log_scale": True},
        {"name": "batch_size", "type": "choice", "values": [128, 256, 512], "value_type": "int", "is_ordered" : True,"sort_values":False}, 
        {"name": "heads", "type": "choice", "values": [1,2], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "heads", "type": "choice", "values": [1], "value_type": "int", "is_ordered" : True,"sort_values":False},
        
        #{"name": "aggregation", "type" : "choice", "values" :["sum", "mean", "max"], "value_type" : "str"}
        {"name": "aggregation", "type" : "choice", "values" :["max"], "value_type" : "str"},
     
    ],
  
    evaluation_function=train_evaluate,
    objective_name='AVG_total_loss',
    arms_per_trial=1,
    minimize = True,
    random_seed = 123,
    total_trials = 20
)

print(best_parameters)
means, covariances = values
print(means)
print(experiment)

[INFO 01-14 20:39:07] ax.service.utils.instantiation: Choice parameter hid contains only one value, converting to a fixed parameter instead.
[INFO 01-14 20:39:07] ax.service.utils.instantiation: Choice parameter layers contains only one value, converting to a fixed parameter instead.
[INFO 01-14 20:39:07] ax.service.utils.instantiation: Choice parameter aggregation contains only one value, converting to a fixed parameter instead.
[INFO 01-14 20:39:07] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='hid', parameter_type=INT, value=128), FixedParameter(name='layers', parameter_type=INT, value=2), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.0001, 0.1], log_scale=True), ChoiceParameter(name='batch_size', parameter_type=INT, values=[128, 256, 512], is_ordered=True, sort_values=False), ChoiceParameter(name='heads', parameter_type=INT, values=[1, 2], is_ordered=True, sort_values=False), FixedParameter(name='aggregation', paramete

{'lr': 0.0001865322718407359, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0


VALIDATION
org:resource: acc 0.0431
Activity: acc 0.5417 macroF1 0.2359
org:resource:role: acc 0.8282
time:timestamp: MAE 3.7173
TOTAL_LOSS: 10.0132
epoch time 39.306875228881836s



KeyboardInterrupt: 

In [ ]:
from ax.service.utils.report_utils import exp_to_df

results = exp_to_df(experiment)

In [ ]:
results.sort_values(by="AVG_total_loss")

In [ ]:
results = results.sort_values(by="AVG_total_loss")

In [ ]:
results.to_csv("BPI17O.csv", sep=",")